In [3]:
import sqlite3
import pandas as pd
import numpy as np

In [4]:
class DatabaseHandler:
    @staticmethod
    def create_database():
        conn = sqlite3.connect('country_data.db') #creates and connects to country_data.db database
        conn.close()
    

    @staticmethod
    def create_table():
        conn = sqlite3.connect('country_data.db')
        c = conn.cursor()
        #create a table called 'songs' with the country, artist, their song, and its rank in that country
        c.execute('''CREATE TABLE IF NOT EXISTS songs (
                    country TEXT,
                    artist_name TEXT,
                    song_name TEXT,
                    rank INTEGER)''')
        conn.commit()
        conn.close()

    @staticmethod
    def insert_data(country, artist_name, song_name, rank):
        conn = sqlite3.connect('country_data.db')
        c = conn.cursor()
        #insert country, artist, and song info into the database
        c.execute("INSERT INTO songs (country, rank, song_name, artist_name) VALUES (?, ?, ?, ?)", (country, artist_name, song_name, rank))
        conn.commit()
        conn.close()


    def populate_db(df = pd.read_csv('country_charts.csv')):
        #These two functions below are called to create the database and table in order to populate that db and table
        DatabaseHandler.create_database()  
        DatabaseHandler.create_table()

        #iterate through pd df and call insert_data function to populate the database
        for index, row in df.iterrows():
            DatabaseHandler.insert_data(row['country'], row['artist_name'], row['song_title'], row['Pos'])


    #function to find songs that are unique to that country
    def find_unique_songs(country):
        conn = sqlite3.connect('country_data.db')
        c = conn.cursor()

        #SQL query that finds distinct song_name from the inputted country
        c.execute("""
        SELECT DISTINCT song_name 
        FROM songs 
        WHERE country = ? 
        AND song_name NOT IN (
            SELECT song_name FROM songs WHERE country != ?
        )
    """, (country, country))
        
        unique_songs = c.fetchall()
        conn.close()
        return unique_songs


In [5]:
DatabaseHandler.populate_db()

In [21]:
@staticmethod
def get_all_country_unique_songs():
    conn = sqlite3.connect('country_data.db')
    c = conn.cursor()
    
    # getting list of all countries in top charts list
    c.execute("SELECT DISTINCT country FROM songs")
    countries = c.fetchall()
    
    # initialize an empty dictionary
    top_songs_by_country = {}

    for country in countries:
        country_name = country[0]
        
        # Retrieve the top songs for the current country
        c.execute("""
            SELECT song_name 
            FROM songs 
            WHERE country = ?
            AND song_name NOT IN (
                SELECT song_name 
                FROM songs 
                WHERE country != ?
            ) 
            ORDER BY rank ASC
        """, (country_name, country_name))
        top_songs = c.fetchall()
        
        # Store the top songs in the dictionary
        top_songs_by_country[country_name] = [song[0] for song in top_songs]

    conn.close()
    return top_songs_by_country


In [22]:
get_all_country_unique_songs()

{'United States': [' all of me',
  ' all of me',
  ' all of me',
  ' all of me',
  ' all of me',
  ' all of me',
  ' all of me',
  ' all of me',
  ' Mmhmm',
  ' Mmhmm',
  ' Mmhmm',
  ' Mmhmm',
  ' Mmhmm',
  ' Mmhmm',
  ' Mmhmm',
  ' Mmhmm',
  ' Novacane',
  ' Novacane',
  ' Novacane',
  ' Novacane',
  ' Novacane',
  ' Novacane',
  ' Novacane',
  ' Novacane',
  ' Deeper Well',
  ' Deeper Well',
  ' Deeper Well',
  ' Deeper Well',
  ' Deeper Well',
  ' Deeper Well',
  ' Deeper Well',
  ' Deeper Well',
  ' LOVE. FEAT. ZACARI. (w/ Zacari)',
  ' LOVE. FEAT. ZACARI. (w/ Zacari)',
  ' LOVE. FEAT. ZACARI. (w/ Zacari)',
  ' LOVE. FEAT. ZACARI. (w/ Zacari)',
  ' LOVE. FEAT. ZACARI. (w/ Zacari)',
  ' LOVE. FEAT. ZACARI. (w/ Zacari)',
  ' LOVE. FEAT. ZACARI. (w/ Zacari)',
  ' LOVE. FEAT. ZACARI. (w/ Zacari)',
  " Let's Go",
  " Let's Go",
  " Let's Go",
  " Let's Go",
  " Let's Go",
  " Let's Go",
  " Let's Go",
  " Let's Go",
  ' Let The Light In (w/ Father John Misty)',
  ' Let The Light In (w/ 

# Country Similarity function pseudocode

    Get all country's unique songs
    Make new dictionary of all country names as the key where the values will be the number of songs from the user in the country's top charts

    For each song in user's top songs:
        identify if that song is in any country's unique top charts 
        if so: 
            Add 1 to the dictionary with that country name
        else:
            move on to next song
        
    Return country of max(# of songs in the dictionary)

In [23]:
#function to return the country that matches user's top songs the most
def count_similarity_to_user(user_top_songs):
    # Get all country's unique songs
    top_songs_by_country = get_all_country_unique_songs()
    # Make a new dictionary to store the count of songs from the user in each country's top charts
    country_song_count = {country: 0 for country in top_songs_by_country.keys()}

    for song in user_top_songs:
        # Identify if that song is in any country's unique top charts
        for country, top_songs in top_songs_by_country.items():
            if song in top_songs:
                # Add 1 to the dictionary with that country name
                country_song_count[country] += 1
                break  # No need to continue searching once the song is found in a country's chart

    return max(country_song_count, key=country_song_count.get)